In [1]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

##### Чтобы нули не попадали в логарифм, добавим небольшой довесок alpha, в первом случае будем прибавлять, во втором отнимать, чтобы сбалансировать

In [14]:
def calc_logloss(y, y_pred):
    alpha = 1e-8
    err = - np.mean(y * np.log(y_pred + alpha) + (1.0 - y) * np.log(1.0 - y_pred - alpha))
    return err

In [17]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782634

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [19]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [22]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [32]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st, y

(array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
        [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
          0.        ,  5.        , 10.        ,  1.        ,  2.        ],
        [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
         -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
        [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
          1.        ,  3.        ,  3.        ,  1.        ,  2.        ]]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

In [38]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [39]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        
        if i % (iterations / 10) == 0:
            print(i, W, err)
        
    return W

In [86]:
W = eval_model(X_st, y, iterations=1000000, alpha=1e-3)

100000 [-2.77073816 -0.9958014   0.56651721  3.26761217] 0.4058257611300329
200000 [-4.5902344  -1.03597991 -0.13798412  4.44453591] 0.35332020884368814
300000 [-5.90935598 -1.08770924 -0.62102637  5.36106613] 0.32500474979932853
400000 [-6.96067626 -1.14367777 -0.98671703  6.13008459] 0.3066194525266598
500000 [-7.85435403 -1.19931706 -1.28727759  6.80516592] 0.29312159266860577
600000 [-8.64671138 -1.25343618 -1.54766856  7.4163573 ] 0.2823914631217132
700000 [-9.36908635 -1.3058213  -1.78112068  7.98147481] 0.27340254195729696
800000 [-10.04018803  -1.35652736  -1.99528145   8.51163508] 0.26560103276748476
900000 [-10.67184112  -1.40567324  -2.19487692   9.01410034] 0.25866224524177717
1000000 [-11.27187924  -1.45338127  -2.38298732   9.49381115] 0.2523828118375754


### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [87]:
def calc_pred_proba(W, X):
    m = X.shape[1]
    
    y_pred_proba = np.zeros((1, m))
    w = W.reshape(X.shape[0], 1)
    
    z = sigmoid(np.dot(w.T, X))
    
    #За порог отнесения к классу 1 примем вероятность > 0.5
    for i in range(z.shape[1]):
        if (z[:,i] > 0.5): 
            y_pred_proba[:, i] = 1 
    
    return 100.0 - np.mean(np.abs(y_pred_proba - y)*100.0)

In [88]:
p1 = calc_pred_proba(W, X_st)
p1

90.0

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [89]:
def calc_pred(W, X):
    m = X.shape[1]
    
    y_pred = np.zeros((1, m))
    w = W.reshape(X.shape[0], 1)
    
    z = sigmoid(np.dot(w.T, X))
    
    #За порог отнесения к тому или иному классу примем вероятность > 0.5
    for i in range(z.shape[1]):
        if (z[:,i] > 0.5): 
            y_pred[:, i] = 1 
        else:
            y_pred[:, i] = 0
    
    return y_pred

In [90]:
y_pred_proba = calc_pred(W, X_st)
y_pred_proba

array([[0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]])

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [100]:
accuracy = np.mean(y_pred_proba == y)
accuracy

0.9

In [101]:
y_pred_proba, y

(array([[0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

In [102]:
TP = 4
FP = 0
FN = 1
TN = 5

In [103]:
precision = TP / (TP + FP)
precision

1.0

In [104]:
recall = TP / (TP + FN)
recall

0.8

In [105]:
F = (2 * precision * recall) / (precision + recall)
F

0.888888888888889

### 6. Могла ли модель переобучиться? Почему?

Могла, если было неверно выбранно число alpha и модель проскочила локальный максимум